# Parallelization Testing   

The purpose of this notebook is to work through how to parallelize evolutionary algorithms using the DEAP framework.  

This notebook will use the one max tutorial from the DEAP documentation.

In [31]:
import random

from deap import base
from deap import creator
from deap import tools

#Scoop for parallelization
from scoop import futures

toolbox.register("map", futures.map)

In [45]:
#Create fitnessmax class
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

#Create individual class
creator.create("Individual", list, fitness=creator.FitnessMax)

In [46]:
#Create toolbox which will hold methods
toolbox = base.Toolbox()

# Attribute generator 
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers that will create individuals and populations
#Individuals are lists of 100 integers that will be zero or 1 as defined by the function attr_bool
toolbox.register("individual", tools.initRepeat, creator.Individual,toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [47]:
#Create the evaluation function
#We want to work towards having one individual with all ones, i.e. a sum of 100
def evalOneMax(individual):
    return sum(individual)

In [48]:
#Create the genetic operators and put them in the toolbox
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [68]:
#Create evolution function in the function main()
def main(CXPB = 0.1, MUTPB = 0.3):
    #Create the population of 300 individuals
    pop = toolbox.population(n=300)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = [fit]
    
    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]
    
    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while max(fits) < 100 and g < 1000:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
                
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = [fit]
            
        #Replace the old population by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
    

In [69]:
main()

-- Generation 1 --
  Min 42.0
  Max 61.0
  Avg 53.7366666667
  Std 3.84629547602
-- Generation 2 --
  Min 45.0
  Max 65.0
  Avg 56.9566666667
  Std 2.77755087962
-- Generation 3 --
  Min 49.0
  Max 65.0
  Avg 58.7766666667
  Std 2.34096608737
-- Generation 4 --
  Min 53.0
  Max 66.0
  Avg 60.13
  Std 2.14003894046
-- Generation 5 --
  Min 51.0
  Max 68.0
  Avg 61.3733333333
  Std 2.25402356293
-- Generation 6 --
  Min 56.0
  Max 69.0
  Avg 62.82
  Std 2.16508660335
-- Generation 7 --
  Min 56.0
  Max 70.0
  Avg 63.96
  Std 2.21924912226
-- Generation 8 --
  Min 59.0
  Max 72.0
  Avg 65.3866666667
  Std 2.18414488734
-- Generation 9 --
  Min 59.0
  Max 73.0
  Avg 66.79
  Std 2.20436687812
-- Generation 10 --
  Min 58.0
  Max 76.0
  Avg 67.9366666667
  Std 2.42198036509
-- Generation 11 --
  Min 63.0
  Max 78.0
  Avg 69.28
  Std 2.20036360632
-- Generation 12 --
  Min 63.0
  Max 78.0
  Avg 70.26
  Std 2.08783779702
-- Generation 13 --
  Min 63.0
  Max 78.0
  Avg 70.9833333333
  Std 2.350